In [7]:
# stdlib imports
from datetime import datetime
from time import time

# Third party imports
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth
import pandas as pd

# Local imports
from libcomcat.dataframes import get_history_data_frame
from libcomcat.search import (get_event_by_id, get_authoritative_info, search)

Query to the API

In [44]:
# VERY approximate Chile bounds
bounds = [-86.748, -66.885, -54.858, -16.774]

# magnitude range
minmag = 3.0
maxmag = 9.9

data = []
for i in range(0,17):
    stime = datetime(2000+i,1,1)
    etime = datetime(2001+i,1,1)
    eventlist = search(starttime=stime,
                  endtime=etime,
                  minlatitude=bounds[2],
                  maxlatitude=bounds[3],
                  minlongitude=bounds[0],
                  maxlongitude=bounds[1],
                  minmagnitude=minmag,
                  maxmagnitude=maxmag)
    data = data + eventlist
    
manipulate = [event.toDict() for event in data]

functions to parse data

In [45]:
def date_extracter(date):
    formatted = date.strftime("%d-%m-%Y (%H:%M:%S.%f)")
    string_subsect = formatted.split(" ")
    # Date formatting
    date = string_subsect[0]
    date = date.split("-")
    #print(date)
    # Hour formatting
    hour = string_subsect[1]
    hour = hour[1:-2] # To cut the parenthesis
    hour = hour.split(":")
    hour[2] = hour[2][0:2]
    # Proper parsing
    parsed = list()
    for item in date:
        parsed.append(item)
    for item in hour:
        parsed.append(item)
    #print(parsed)
    return parsed
    #return date

def event_generator(date, lat, lon, depth, magnitude):
    #print(date)
    day = date[0]
    month = date[1]
    year = date[2]
    hour = date[3]
    minute = date[4]
    second = date[5]
    event = [year, month, day, hour, minute, second, lat, lon, depth, magnitude]
    return event

def event_extracter(events):
    event_formatted = list()
    for event in events:
        #print(event)
        for key, value in event.items():
            #print(key, value)
            if key == "time":
                #print("ASDF")
                #print(type(value))
                date = date_extracter(value)
                #print("DATE")
                #print(date)
            if key == "latitude":
                lat = value
                #print("LAT")
            if key == "longitude":
                lon = value
                #print("LON")
            if key == "depth":
                depth = value
                #print("DEPTH")
            if key == "magnitude":
                magnitude = value
                #print("MAG")
            #print(date)
            #print(lat)
            #print(lon)
            #print(depth)
            #print(magnitude)
        ev = event_generator(date, lat, lon, depth, magnitude)
        event_formatted.append(ev)
    return event_formatted

data formatted

In [46]:
event_formatted = event_extracter(manipulate)

visualising in dataframe

In [47]:
df = pd.DataFrame(event_formatted, columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "Latitude", "Longitude", "Depth", "Magnitude"])
df

,Year,Month,Day,Hour,Minute,Second,Latitude,Longitude,Depth,Magnitude
0,2000,01,01,04,30,10,-26.0120,-68.8030,105.00,4.5
1,2000,01,01,16,01,13,-22.2140,-70.1910,67.20,4.4
2,2000,01,02,02,13,28,-17.3460,-69.2770,161.60,3.7
3,2000,01,03,04,03,14,-21.1100,-68.4240,134.40,4.0
4,2000,01,06,21,37,49,-18.3340,-69.4460,143.20,4.6
...,...,...,...,...,...,...,...,...,...,...
23928,2016,12,28,23,51,54,-43.1386,-74.4594,19.42,4.1
23929,2016,12,29,08,31,06,-21.5902,-68.3630,115.15,4.6
23930,2016,12,29,08,50,21,-33.6842,-71.5737,43.26,4.4
23931,2016,12,30,15,48,24,-31.6078,-68.6146,107.61,4.5


Saving to a file

In [48]:
pickled_data = open("data24k.pkl", "wb")
pickle.dump(event_formatted, pickled_data)